In [1]:
import os
import re
import json
import shutil
import re
import logging
from math import radians, cos, sin, asin, sqrt

In [2]:
import pandas as pd
# from tqdm import tqdm
import numpy as np
import math

from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.model_selection import KFold

import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install --upgrade pip
!pip install lightgbm
import lightgbm as lgb

    100% |████████████████████████████████| 1.5MB 79.2MB/s ta 0:00:01�███████████████████████████▊ | 1.4MB 80.6MB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 1.2 MB 85.1 MB/s eta 0:00:01


In [4]:
!pip install geopandas
!pip install descartes
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [37]:
# baseline只用到gps定位数据，即train_gps_path
train_gps_path = 'train0714.csv'
test_data_path = 'R2_ATest0711.csv'
order_data_path = 'loadingevents.csv'
port_data_path = 'port_fixed_0612.csv'

In [38]:
def get_data(data, mode = 'train'):
    assert mode=='train' or mode=='test'
    
    # data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['loadingOrder'] = data['loadingOrder'].astype(str)
    data['latitude'] = data['latitude'].astype(float)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float)

    data["TRANSPORT_TRACE"] = data['TRANSPORT_TRACE'].astype(str)
    # data["Start"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[0])
    if mode=='train':
        data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True)
        # data["End"] = data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
        # data['StartEnd'] = data['Start'].values+'-'+data["End"].values
    elif mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)

    print('========== Finish Collecting Data ==========')
    return data

In [39]:
test_data = pd.read_csv('/home/ma-user/work/FuSai/data/'+test_data_path)
course_list = list(test_data["TRANSPORT_TRACE"].unique())
test_data["Port_list"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-"))
test_data["End"] = test_data["TRANSPORT_TRACE"].apply(lambda x: x.split("-")[-1])
end_list = list(test_data["End"].unique())
timestamp = test_data['timestamp']
test_data = get_data(test_data, mode='test')

========== Finish Collecting Data ==========


In [11]:
port_list = []
for i in range(len(np.unique(test_data['Port_list'].values))):
    port_list.extend(np.unique(test_data['Port_list'].values)[i])
port_list = list(np.unique(sorted(port_list)))

In [9]:
sorted(end_list)

NameError: name 'end_list' is not defined

In [12]:
order_list = test_data['loadingOrder'].unique()
order_length = []
for i in range(len(order_list)):
    dff = test_data[test_data['loadingOrder'] == order_list[i]].shape[0]
    order_length.append(dff)

port_groups = np.array((['CNDCB','CNHKG','CNNBG','CNNSA','CNSGH','CNSHA','CNSHK','CNXAM','CNYTN','HKHKG','HON','TWKHH'],\
                        
                ['MYTPP','SGSIN'],\
                ['EGPSD','EGPSE','EGSUE','EGSUZ'],\
                ['MUPLU','NAWVB','ZACPT','ZADUR'],\
                ['PABLB','PACOL','PAMIT','PAONX'],\
                ['AUBNE','NZAKL'],\
                        
                ['KHKOS','PHBTG','THLCH','VNVUT'],\
               ['CNTAO','KRINC','KRPUS','JPHKT','JPTYO','RUVVO'],\
                ['CLSAI','CLSAN','CLVAP'],\
                
                ['CAVAN','CRLIO','GTIZ4','GTPRQ','HNPCR','MXLZC','MXZLO','SVAQJ'],\
               ['ANPHI','COBUN','COCTG','DOCAU','DOHAI','GPPTP','JMKIN','TTPOS'],\

                ['BDCGP','IDJKT','MYPKG'],\
                
              ['ARBUE','ARENA','BRNVT','BRPNG','BRRIO','BRSSZ','UYMVD'],\
                ['BJCOO','CIABJ','CMKBI','CVRAI','GALBV','GHTEM','GMBJL','CGPNR','NGAPP','SNDKR','TGLFW'],\
                        
              ['ESALG','ESBCN','ESVAL','FRFOS','FRMRS','GRPIR','HRRIJ','HUNGARY','LBBEY','MTMLA','SIKOP'],\
                ['DZALG','ITGOA','ITSPE','MACAS','MATNG','TNTUN'],\
                        
               ['CAHAL','DEHAM','FRLEH','GBFXT','GBSOU','LTKLJ','NLRTM','RTM'],\
                        
              ['AEAUH','AEJEA','BHBAH','KWKWI','LKCMB','OMSOH','QAHMD','SADMM'],\
                ['INMAA','INMUN','INNSA','PKBQM','PKQCT']))

adjacant_number = np.array(([0],\
                [0,1],[1,2],[1,3],[4,9,10],[0,5],\
                [0,6],[0,7],[5,8],\
                [0,7,9],[9,10],\
                [1,11],[3,12],[2,3,13,14],[2,14],[2,15],\
                [1,2,13,14,16],[1,18,17],[1,18]))

In [15]:
test_schedule

,loadingOrder,number,start_time,start_longitude,start_latitude,end_time,end_longitude,end_latitude,begin_longitude,begin_latitude,begin_port,target_longitude,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest
0,AC437723355280,1,0.0,113.895823,22.383463,905050.0,162.527333,-21.223300,113.863058,22.559462,CNSHK,-71.642993,-33.030843,CLVAP,7233.013003,26.646341,0.017964,12958.070122,7162.936,11684.294
1,AC860038925693,1,0.0,114.290705,22.564435,455150.0,103.092600,1.483327,114.275347,22.577700,CNYTN,-5.812980,35.788207,MATNG,2812.895522,24.868421,0.025641,22206.447368,2637.942,11601.618
2,AR701843002140,1,0.0,113.694225,22.630953,2000.0,113.737135,22.522195,113.863058,22.559462,CNSHK,103.704620,1.303141,SGSIN,12.874376,22.210526,0.000000,15890.000000,12.871,2596.698
3,AT407433613767,1,0.0,104.196312,1.287627,849960.0,43.505220,12.552002,114.139709,22.419915,HKHKG,4.885655,43.430234,FRFOS,7006.465551,28.855263,0.146067,27262.789474,6799.112,5038.529
4,AV639724236766,1,0.0,113.894683,22.451200,17098.0,114.265950,21.757183,113.863058,22.559462,CNSHK,80.282252,13.066265,INMAA,110.163716,22.435714,0.000000,15252.142857,86.130,3723.837
5,BA498843975994,1,0.0,113.899117,22.461683,77686.0,118.285350,24.225433,113.863058,22.559462,CNSHK,130.410386,33.660845,JPHKT,582.093519,25.831818,0.000000,10359.909091,488.827,1576.557
6,BB470439135271,7,0.0,129.496267,34.140347,85815.0,135.931387,38.613280,114.275347,22.577700,CNYTN,-89.829556,13.581498,SVAQJ,760.400176,31.725566,0.000000,4876.803081,760.735,13853.655
7,BE790761427541,1,0.0,113.883267,22.444100,36003.0,113.890616,20.098216,113.863058,22.559462,CNSHK,50.618590,26.135392,BHBAH,310.298333,28.740741,0.000000,15896.543210,260.852,6444.569
8,BE898272362291,1,0.0,114.273142,22.563410,146650.0,120.330858,22.550167,114.275347,22.577700,CNYTN,-104.305571,19.085961,MXZLO,643.798916,31.384615,0.717391,10423.076923,622.017,13311.457
9,BE929753510083,1,0.0,114.285403,22.576393,238450.0,122.399372,30.541828,114.275347,22.577700,CNYTN,-79.882990,9.352609,PAONX,1426.588993,30.215909,0.000000,12477.727273,1197.382,14981.314


In [23]:
train_schedule = pd.read_csv('/home/ma-user/work/FuSai/data/train_schedule.csv')
test_schedule = pd.read_csv('/home/ma-user/work/FuSai/data/test_schedule.csv')

In [24]:
test_schedule = test_schedule.sort_values(by=['loadingOrder'],ascending = [True])

train_schedule['rho'] = train_schedule['ds']/train_schedule['route_dist']
test_schedule['rho'] = test_schedule['ds']/test_schedule['route_dist']
train_schedule = train_schedule[train_schedule['label'] > 0]

In [43]:
train_schedule = train_schedule.fillna(value={'speed':0.,'direction':0.,'rho':1.})
test_schedule = test_schedule.fillna(value={'speed':0.,'direction':0.,'rho':1.,'stop_fraction':1.})

In [27]:
def build_model(train, test, n_splits, pred, residual_list, seed=1080, is_shuffle=True):
    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    # Kfold
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    kf_way = fold.split(train[pred])
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression_l2',
        'num_leaves': 31,
        'metric': 'mse',
        'feature_fraction': 0.6,
        'bagging_fraction': 0.7,
        'bagging_freq': 6,
        'seed': 2,
        'random_state': 32,
        'lambda_l2': 0.5,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 15,
        'nthread': 8,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        train_x, train_y = train[pred].iloc[train_idx], train[residual_list[-2]].iloc[train_idx]
        valid_x, valid_y = train[pred].iloc[valid_idx], train[residual_list[-2]].iloc[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=n_train,
            num_boost_round=20000,
            valid_sets=[n_valid],
            early_stopping_rounds=100,
            verbose_eval=100
        )
        

        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test[pred], num_iteration=clf.best_iteration)/fold.n_splits
    train[residual_list[-1]] = train_pred-train[residual_list[-2]].values
    
    test[residual_list[-2]] = test_pred
    
    return test[residual_list[0:-1]], train

In [20]:
len(train_schedule['loadingOrder'].unique())

10233

In [21]:
port_groups[i]

['CNDCB',
 'CNHKG',
 'CNNBG',
 'CNNSA',
 'CNSGH',
 'CNSHA',
 'CNSHK',
 'CNXAM',
 'CNYTN',
 'HKHKG',
 'HON',
 'TWKHH']

In [70]:
train_set = train_schedule[((train_schedule['end_port'].isin(port_groups[i]))&(train_schedule['start_port'].isin(port_groups[0])))]

In [120]:
train_set = train_schedule[((train_schedule['end_port'].isin(port_groups[i]))&(train_schedule['target_port'].isin(port_groups[i]))&\
                                    (train_schedule['start_port'].isin(port_groups[0])))]
train_set

,loadingOrder,number,start_time,start_longitude,start_latitude,start_port,end_time,end_longitude,end_latitude,end_port,...,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest,label,rho
1287,AU372091858162,9,2017125.0,118.062781,24.471864,CNXAM,2095217.0,113.888900,22.459591,CNDCB,...,22.419915,HKHKG,621.333147,28.613372,0.000000,24327.558140,480.907,26.152,3.819537e+03,1.292003
1523,AX891281472890,7,2731246.0,120.289275,22.623384,TWKHH,2852212.0,114.118218,22.347969,HON,...,22.419915,HKHKG,656.951452,20.801056,0.000000,26828.285211,634.716,8.300,1.277467e+03,1.035032
1789,BB703632984745,9,1734671.0,120.289275,22.623384,TWKHH,1822977.0,114.123547,22.330680,HON,...,22.419915,HKHKG,660.340384,24.625641,0.000000,26894.128205,634.305,10.061,1.139335e+03,1.041046
3333,BN609583117938,1,501.0,114.275347,22.577700,CNYTN,105998.0,120.290403,22.537098,TWKHH,...,37.437373,CNYTN,626.996240,20.746753,0.000000,10337.814935,617.649,1763.431,5.152713e+06,1.015134
4194,BU717719720072,1,1810.0,113.870200,22.538085,CNDCB,125089.0,120.330385,22.550855,TWKHH,...,17.486323,CNSHK,687.854284,21.442465,0.000000,9358.307350,663.396,919.567,3.688727e+05,1.036868
4687,BY694518769931,1,447.0,114.275347,22.577700,CNYTN,128369.0,120.246166,22.539902,TWKHH,...,40.760233,CNYTN,612.953166,15.192661,0.000000,9659.853211,613.099,8436.324,5.300739e+06,0.999762
6024,CI718949315772,1,51595.0,114.110755,22.335179,HON,154394.0,118.076695,24.508538,CNXAM,...,22.419915,HKHKG,665.506825,32.771402,0.000000,14847.930386,471.284,463.854,5.095654e+05,1.412114
6949,CO789628821908,13,5170893.0,122.056860,29.887199,CNNBG,5373066.0,114.116665,22.329668,HON,...,22.419915,HKHKG,1358.848191,24.333333,0.000000,22774.983498,1154.557,10.311,1.650866e+03,1.176943
8948,DE262086771416,9,1734671.0,120.289275,22.623384,TWKHH,1822977.0,114.123547,22.330680,HON,...,22.419915,HKHKG,660.340384,24.625641,0.000000,26894.128205,634.305,10.061,1.139335e+03,1.041046
8958,DE269795010074,1,879.0,114.275347,22.577700,CNYTN,100952.0,120.292763,22.538178,TWKHH,...,22.419915,HKHKG,632.020038,22.327014,0.000000,9037.298578,617.888,632.296,5.304575e+06,1.022872


In [148]:
i = 1

In [149]:
train_set = train_schedule[((train_schedule['end_port'].isin(port_groups[i]))&(train_schedule['target_port'].isin(port_groups[i]))|\
                                    (train_schedule['start_port'].isin(port_groups[i])))]
train_set

,loadingOrder,number,start_time,start_longitude,start_latitude,start_port,end_time,end_longitude,end_latitude,end_port,...,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest,label,rho
28,AA642567891255,3,437327.0,103.704620,1.303141,SGSIN,1203275.0,54.924678,25.163411,AEJEA,...,26.135392,BHBAH,6473.210073,25.833333,0.000000,27217.707602,5860.415,444.931,6.772654e+04,1.104565
45,AB232279245750,3,339710.0,103.704620,1.303141,SGSIN,1262267.0,32.560347,29.831548,EGSUZ,...,51.963177,GBFXT,9219.247452,34.607251,0.000000,29330.666667,8122.357,3549.363,7.490406e+05,1.135046
61,AB341124011989,3,986153.0,103.545456,1.399416,MYTPP,1985361.0,32.560105,29.832707,EGSUZ,...,3.034709,MYPKG,9152.859966,30.811102,0.000000,29624.507042,8101.719,7800.156,2.044325e+06,1.129743
68,AB470160617552,3,362748.0,103.545456,1.399416,MYTPP,1291385.0,32.569074,29.828651,EGSUZ,...,3.034709,MYPKG,9226.300957,28.802062,0.000000,26965.901031,8100.797,7799.235,2.100334e+06,1.138937
75,AB503189741912,3,456417.0,103.704620,1.303141,SGSIN,2384047.0,-43.203011,-22.873811,BRRIO,...,-23.954513,BRSSZ,16545.961945,30.198709,0.000000,26837.044476,15713.067,336.554,1.166570e+05,1.053007
82,AB583454790935,3,403164.0,103.545456,1.399416,MYTPP,566306.0,75.964838,9.049682,GHOST2,...,-23.954513,BRSSZ,1655.677130,35.649573,0.000000,29183.102564,3167.850,13682.985,2.158064e+06,0.522650
91,AB614679152621,3,351123.0,103.704620,1.303141,SGSIN,1308298.0,32.570252,29.737546,EGSUZ,...,39.460366,ESVAL,9133.703562,31.559991,0.000000,28766.827498,8119.873,3178.130,5.895590e+05,1.124858
102,AB690600150591,3,214676.0,103.704620,1.303141,SGSIN,258926.0,101.281240,2.912141,MYPKG,...,-29.868304,ZADUR,370.303756,30.300000,0.000000,26214.628571,323.295,8281.495,1.308528e+06,1.145405
113,AB895237846468,3,530291.0,103.545456,1.399416,MYTPP,1435807.0,32.561794,29.818730,EGSUZ,...,14.690892,SNDKR,9157.057412,33.763786,0.000000,29734.709465,8101.329,5366.877,1.084491e+06,1.130315
122,AB900832754404,3,459458.0,103.704620,1.303141,SGSIN,1456568.0,32.557202,29.809208,EGSUZ,...,14.690892,SNDKR,9140.867189,31.341360,0.000000,29642.232295,8122.291,5366.311,1.022803e+06,1.125405


In [151]:
test_set = test_schedule[((train_schedule['end_port'].isin(port_groups[i])))|\
                                  (test_schedule['begin_port'].isin(port_groups[i]))]
test_set

,loadingOrder,number,start_time,start_longitude,start_latitude,end_time,end_longitude,end_latitude,begin_longitude,begin_latitude,...,target_longitude,target_latitude,target_port,ds,speed,stop_fraction,direction,route_dist,dist_to_dest,rho
26,DM496581722833,1,0.0,1.287470,6.144010,28929.0,1.302262,6.128752,113.653433,22.694213,...,9.906216,2.939002,CMKBI,3.118725,7.900000,0.886364,14825.000000,2.356,1017.417,1.323737
43,FZ214726756350,1,0.0,87.117378,0.543337,844810.0,29.775475,-32.900300,114.275347,22.577700,...,-58.356770,-34.627862,ARENA,7052.695508,29.663866,0.008333,23956.974790,7048.705,7857.015,1.000566
59,HQ193237808500,1,0.0,113.879350,22.458848,8210.0,113.877398,22.457192,113.863058,22.559462,...,103.704620,1.303141,SGSIN,0.327194,2.000000,0.928571,19010.000000,0.272,2596.591,1.202920
66,IV490066651689,1,0.0,113.925250,22.355693,308220.0,104.554713,1.503073,113.863058,22.559462,...,67.327480,24.766207,PKQCT,2656.901372,44.633333,0.000000,18274.716667,2530.321,4758.978,1.050025
73,JK194826361312,1,0.0,113.898700,22.455833,21200.0,115.325950,22.231867,113.863058,22.559462,...,139.792252,35.616509,JPTYO,175.397293,26.830000,0.000000,11815.000000,148.884,2795.518,1.178080
80,KC730898618404,1,0.0,102.090293,2.106948,86215.0,96.717632,5.642327,114.275347,22.577700,...,-58.356770,-34.627862,ARENA,721.704802,30.840149,0.000000,30466.403346,713.931,15899.110,1.010889
89,LK919030439899,1,0.0,54.018295,16.948262,25250.0,55.964582,16.063627,13.728527,45.537061,...,114.275347,22.577700,CNYTN,231.501465,29.750000,0.000000,11280.250000,229.631,6125.256,1.008146
100,NB639332663495,1,0.0,113.898235,22.381008,2360.0,114.051765,22.356727,113.863058,22.559462,...,103.704620,1.303141,SGSIN,17.004426,23.600000,0.000000,9166.000000,16.016,2594.667,1.061715
111,OA206903187361,1,0.0,113.881085,22.448203,1760.0,113.897010,22.384398,113.863058,22.559462,...,103.545456,1.399416,MYTPP,7.313302,14.500000,0.000000,16365.500000,7.281,2587.899,1.004436
120,OT727256058927,1,0.0,34.213678,27.402342,85958.0,32.571037,29.836490,113.863058,22.559462,...,8.946256,44.405650,ITGOA,325.542217,25.109756,0.507508,30015.182927,314.579,1417.717,1.034850


In [24]:
i = 1
test_set = test_schedule[((test_schedule['target_port'].isin(port_groups[i])))|(test_schedule['begin_port'].isin(port_groups[i]))]
test_set.shape[0]

49

In [21]:
i = 1
relevant_port = np.unique(port_groups[adj_cluster[1]])
relevant_port

array([list(['CNDCB', 'CNHKG', 'CNNBG', 'CNNSA', 'CNSGH', 'CNSHA', 'CNSHK', 'CNXAM', 'CNYTN', 'HKHKG', 'HON', 'TWKHH']),
       list(['MYTPP', 'SGSIN'])], dtype=object)

res = None
feature = ['loadingOrder','number','end_time','rho','speed','direction','dist_to_dest','stop_fraction','label']

for i in range(0,len(port_groups)):
    train_set = train_schedule[train_schedule['start_port'] != train_schedule['end_port']]
    if i == 0:
        train_set = train_schedule[(train_schedule['end_port'] == 'TWKHH')&(train_schedule['start_port'].isin(['CNYTN','CNSHK','CNDCB']))]
        test_set = test_schedule[(test_schedule['target_port'].isin(port_groups[i])&(test_schedule['begin_port'].isin(port_groups[i])))]
    elif i < 3:
        train_set = train_schedule[((train_schedule['end_port'].isin(port_groups[i]))&(train_schedule['target_port'].isin(port_groups[i]))|\
                                    (train_schedule['start_port'].isin(port_groups[i])))]
        test_set = test_schedule[((test_schedule['target_port'].isin(port_groups[i])))|(test_schedule['begin_port'].isin(port_groups[i]))]
    else:
        train_set = train_schedule[((train_schedule['end_port'].isin(port_groups[i]))&(train_schedule['target_port'].isin(port_groups[i]))|\
                                    (train_schedule['start_port'].isin(port_groups[i])))]
        test_set = test_schedule[((test_schedule['target_port'].isin(port_groups[i])))|(test_schedule['begin_port'].isin(port_groups[i]))]
    train_set['label'] = 3600*train_set['dist_to_dest']/(train_set['speed'] * (1-train_set['stop_fraction']**2))
    train_set = train_set[feature]
    test_set = test_set[feature[0:-1]]
    print("reading the route:", port_groups[i],',the size of the test =',test_set.shape,',the size of train =',train_set.shape)

    # ETA = np.nan*np.zeros((test_set.shape[0],21*4))
    train_orders = train_set['loadingOrder'].unique()

    nk, nj = min(2 * len(train_orders) + 1, 10), 10
    ETA = np.nan*np.zeros((test_set.shape[0],nk))
    # j = 1
    # print('========== train by',byorder,'orders ============')
    # torder = train_orders[byorder*j:min(byorder*(j+1),len(train_orders))]
    # train_sett = train_set[train_set['loadingOrder'].isin(torder)]
    for k in range(nk):
        residual_list = ['loadingOrder','label']
        for j in range(nj):
            residual = 'residual_'+str(j+1)
            residual_list.append(residual)
            features = [c for c in feature if c not in residual_list]
            result, train_set = build_model(train_set, test_set, np.min([train_set.shape[0],25]), features, residual_list, seed=1080+7*k, is_shuffle=True)
            maxmin = np.array([train_set[residual_list[-1]].max(),train_set[residual_list[-1]].min()])
            zscore = (maxmin - train_set[residual_list[-1]].mean())/train_set[residual_list[-1]].std()
            print('========== i =',i,',j =',j,',k =',k,'the zscore is',zscore,'==========')
            print('the size of train is ',train_set.shape)
            while (train_set.shape[0] >= 10) and ((zscore[0] > 3.5) or (zscore[1] < -3.5)):
                if zscore[0] > 3.:
                    train_set = train_set[~(train_set[residual_list[-1]] == maxmin[0])]
                if zscore[1] < -3.:
                    train_set = train_set[~(train_set[residual_list[-1]] == maxmin[1])]
                maxmin = np.array([train_set[residual_list[-1]].max(),train_set[residual_list[-1]].min()])
                zscore = (maxmin - train_set[residual_list[-1]].mean())/train_set[residual_list[-1]].std()
        print(train_set[residual_list[2::]])
        train_set = train_set.drop(residual_list[2::],axis=1)
        result_std = result.std(axis=1)
        if np.shape(np.where(result.columns[1::]==result[1::].std().idxmin(axis=1))[0])[0] > 0:
            res_ind = np.where(result.columns[1::]==result.std().idxmin(axis=1))[0][0]
        else:
            res_ind = 4
        print('==========',result[1::].std().idxmin(axis=1),',',res_ind,'has the smallest standard deviation.')#' within the route',course_list[i],'==========')
        ETA[:,k] = test_set['end_time'] + result[residual_list[1:res_ind+2]].sum(axis=1) 
    ETA_50 = np.percentile(ETA,50,axis=1)
    print(ETA_50)
    result['label'] = ETA_50
    res = pd.concat([res,result])
    print('==========',port_groups[i],', the shape is:',res.shape,'completed training ==========')

## 不分路线训练

In [30]:
feature = ['loadingOrder','number','end_time','rho','speed','direction','dist_to_dest','stop_fraction','label']

train_set = train_schedule[train_schedule['start_port'] != train_schedule['end_port']]
train_set['label'] = 3600*train_set['dist_to_dest']/(train_set['speed'] * (1-train_set['stop_fraction']**2))
train_set = train_set[feature]
test_set = test_schedule[feature[0:-1]]
print('the size of the test =',test_set.shape,',the size of train =',train_set.shape)

# ETA = np.nan*np.zeros((test_set.shape[0],21*4))
train_orders = train_set['loadingOrder'].unique()

nk, nj = min(2 * len(train_orders) + 1, 10), 10
ETA = np.nan*np.zeros((test_set.shape[0],nk))
# j = 1
# print('========== train by',byorder,'orders ============')
# torder = train_orders[byorder*j:min(byorder*(j+1),len(train_orders))]
# train_sett = train_set[train_set['loadingOrder'].isin(torder)]
for k in range(nk):
    residual_list = ['loadingOrder','label']
    for j in range(nj):
        residual = 'residual_'+str(j+1)
        residual_list.append(residual)
        features = [c for c in feature if c not in residual_list]
        result, train_set = build_model(train_set, test_set, np.min([train_set.shape[0],25]), features, residual_list, seed=1080+7*k, is_shuffle=True)
        print('the size of train is ',train_set.shape)
    print(train_set[residual_list[2::]])
    train_set = train_set.drop(residual_list[2::],axis=1)
    result_std = result.std(axis=1)
    if np.shape(np.where(result.columns[1::]==result[1::].std().idxmin(axis=1))[0])[0] > 0:
        res_ind = np.where(result.columns[1::]==result.std().idxmin(axis=1))[0][0]
    else:
        res_ind = 4
    print('==========',result[1::].std().idxmin(axis=1),',',res_ind,'has the smallest standard deviation.')#' within the route',course_list[i],'==========')
    ETA[:,k] = test_set['end_time'] + result[residual_list[1:res_ind+2]].sum(axis=1) 
ETA_50 = np.percentile(ETA,50,axis=1)
print(ETA_50)
result['label'] = ETA_50

the size of the test = (219, 8) ,the size of train = (27583, 9)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.02688e+10
[200]	valid_0's l2: 5.87108e+09
[300]	valid_0's l2: 5.17099e+09
[400]	valid_0's l2: 4.49657e+09
[500]	valid_0's l2: 4.13565e+09
Early stopping, best iteration is:
[491]	valid_0's l2: 3.97689e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.50663e+09
[200]	valid_0's l2: 7.78005e+09
Early stopping, best iteration is:
[186]	valid_0's l2: 6.47131e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.82935e+10
[200]	valid_0's l2: 1.05212e+10
[300]	valid_0's l2: 8.99607e+09
[400]	valid_0's l2: 8.56427e+09
[500]	valid_0's l2: 7.32341e+09
[600]	valid_0's l2: 6.69912e+09
[700]	valid_0's l2: 6.70219e+09
Early stopping, best iteration is:
[628]	valid_0's l2: 5.99605e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.03413e+11
[200]	v

[200]	valid_0's l2: 5.15365e+10
[300]	valid_0's l2: 4.61634e+10
Early stopping, best iteration is:
[257]	valid_0's l2: 4.42309e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.82463e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.93205e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.46705e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 1.76403e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.24683e+09
Early stopping, best iteration is:
[12]	valid_0's l2: 4.61928e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.14243e+10
[200]	valid_0's l2: 4.15506e+10
Early stopping, best iteration is:
[102]	valid_0's l2: 4.06878e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.38415e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.20621e+10
Training until validation scores don't 

Early stopping, best iteration is:
[3]	valid_0's l2: 6.26298e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 8.1219e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 5.85092e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.77521e+10
[200]	valid_0's l2: 2.60449e+10
[300]	valid_0's l2: 2.36739e+10
Early stopping, best iteration is:
[257]	valid_0's l2: 2.29611e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.90618e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 2.93626e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.39345e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 1.7743e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.18813e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.61091e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0'

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 7.96987e+09
Early stopping, best iteration is:
[7]	valid_0's l2: 5.70689e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.74223e+10
Early stopping, best iteration is:
[9]	valid_0's l2: 1.71429e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.33542e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.98579e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.3076e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 1.79317e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.34654e+09
Early stopping, best iteration is:
[30]	valid_0's l2: 4.58029e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.50275e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 4.11561e+10
Training until validation scores don't improve

Early stopping, best iteration is:
[7]	valid_0's l2: 5.63324e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.73656e+10
Early stopping, best iteration is:
[2]	valid_0's l2: 1.66877e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.31713e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.04651e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.43673e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 1.813e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.4482e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.65502e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.55118e+10
Early stopping, best iteration is:
[2]	valid_0's l2: 4.14523e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.58092e+10
Early stopping, best iteration is:
[1]	valid_0's 

...

[500]	valid_0's l2: 2.57144e+10
[600]	valid_0's l2: 2.51113e+10
[700]	valid_0's l2: 2.26861e+10
[800]	valid_0's l2: 2.28846e+10
Early stopping, best iteration is:
[761]	valid_0's l2: 2.18911e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.86083e+09
Early stopping, best iteration is:
[58]	valid_0's l2: 5.48403e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.70858e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.17606e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.87679e+09
[200]	valid_0's l2: 2.65942e+09
Early stopping, best iteration is:
[144]	valid_0's l2: 2.43004e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.94471e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.6869e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.53697e+09
Early stopping, best iteration is:
[1]	va

[100]	valid_0's l2: 1.9158e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 1.74896e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.31138e+10
Early stopping, best iteration is:
[4]	valid_0's l2: 4.56242e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.00955e+10
[200]	valid_0's l2: 1.9359e+10
Early stopping, best iteration is:
[108]	valid_0's l2: 1.90835e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.66264e+09
Early stopping, best iteration is:
[58]	valid_0's l2: 4.49276e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.65474e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.20239e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.7188e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.55356e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0'

[100]	valid_0's l2: 5.56354e+10
Early stopping, best iteration is:
[4]	valid_0's l2: 4.6479e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.91545e+10
[200]	valid_0's l2: 1.82796e+10
[300]	valid_0's l2: 1.74149e+10
[400]	valid_0's l2: 1.54194e+10
[500]	valid_0's l2: 1.51154e+10
Early stopping, best iteration is:
[494]	valid_0's l2: 1.48723e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.23599e+09
Early stopping, best iteration is:
[63]	valid_0's l2: 4.01271e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.71576e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.22563e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 2.9499e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.73774e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.68152e+09
Early stopping, best iteration is:
[1]	valid

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.72599e+10
Early stopping, best iteration is:
[5]	valid_0's l2: 4.68537e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.58946e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.50912e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.02489e+09
Early stopping, best iteration is:
[20]	valid_0's l2: 3.79307e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.71807e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.25214e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.08751e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 2.93208e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.12482e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.01313e+09
Training until validation scores don't improv

[100]	valid_0's l2: 5.73917e+10
Early stopping, best iteration is:
[5]	valid_0's l2: 4.73506e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.63315e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.52653e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.976e+09
Early stopping, best iteration is:
[2]	valid_0's l2: 3.79043e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.75794e+10
Early stopping, best iteration is:
[1]	valid_0's l2: 1.27863e+10
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.41293e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 3.14604e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 5.16752e+09
Early stopping, best iteration is:
[1]	valid_0's l2: 4.14413e+09
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3.01241e+09
Early stopping, 

In [41]:
res = result

In [42]:
res1 = res.sort_values(by='loadingOrder')
res = res.drop_duplicates(subset=['loadingOrder'],keep='last')
ETA_replace = []
for j in range(order_list.shape[0]):
    # ETA_replace.extend(list(np.repeat(test_schedule['label'].iloc[j],order_length[j])))
    ETA_replace.extend(list(np.repeat(res1['label'].iloc[j],order_length[j])))
test_data = test_data.sort_values(by = ['loadingOrder'], ascending = [True])
test_data['ETA'] = ETA_replace
test_data['ETA'] = (test_data['onboardDate'] + test_data['ETA'].apply(lambda x:pd.Timedelta(seconds=x))).apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data = test_data.sort_values(by = ['timestamp'], ascending = [True])

In [230]:
test_data

,loadingOrder,label,residual_1,residual_2,residual_3,residual_4,residual_5,residual_6,residual_7,residual_8,residual_9
157,SZ109188860665,4.896507e+06,-17506.729150,740.518032,-10800.022387,12231.729104,-12743.808062,-1782.625652,-371.944222,-3800.120248,3564.307622
5,BA498843975994,4.887269e+05,-189.784410,-1442.618356,-1500.339008,-530.968307,165.178695,-79.323366,-377.832036,783.119766,155.133448
47,GL652883735085,3.944320e+05,1214.044322,-2951.873425,-1355.812542,-719.294168,-368.969347,54.923179,-1123.134042,1514.359844,-409.300356
73,JK194826361312,5.242284e+05,8807.317000,-3552.586524,-335.374873,-1384.118609,239.796889,-393.896924,-394.422601,783.119766,253.411917
82,KL671073399431,2.038675e+05,-12841.180981,-1648.570793,748.020830,-668.428433,951.505769,-505.014309,464.203224,-397.672266,492.761201
187,VZ162290304055,3.539644e+05,-15245.526105,1837.403014,-3193.040298,509.549657,-837.787273,842.136467,-1406.605784,1884.946089,-906.744841
203,YJ156074308770,3.132092e+05,-3012.433207,-3953.996928,-193.950320,-2093.624956,1516.516718,-1423.842981,974.523598,-1302.094442,2308.169863
6,BB470439135271,2.536221e+06,11445.279570,3754.183012,2528.390320,-223.038463,995.011845,281.701420,45.235536,2845.628746,157.941999
8,BE898272362291,1.939600e+06,17618.148361,5176.891767,1162.775405,522.817971,329.580039,-482.129228,1375.807245,-1083.056845,506.416147
9,BE929753510083,2.051057e+06,7370.219588,3902.585929,1520.409242,436.784836,-406.310352,-81.921309,1007.194807,527.443661,649.565859


## 手动修改ETA

In [43]:
test_data['ETA'][test_data['loadingOrder'] == 'FA712454830300'] = pd.datetime(2020,2,4,5,7,11).strftime('%Y/%m/%d  %H:%M:%S')

In [44]:
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = timestamp.astype(str)
# 整理columns顺序
results = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]
results.to_csv('/home/ma-user/work/FuSai/result072403.csv', index=False)

In [112]:
results = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]
results.to_csv('/home/ma-user/work/FuSai/result072401.csv', index=False)